In [1]:
import tensorflow as tf
import os
from dataclasses import dataclass
from pathlib import Path 
from CNN_Classifier.constants import *
from CNN_Classifier.utils import read_yaml, create_directories, save_json
import mlflow
import mlflow.keras
from urllib.parse import urlparse

os.chdir("../")

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/VaheC/CNN_Classifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "VaheC"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f6eb840925a7b3b324b5e7cd782c89a321559687"

In [2]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [3]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path 
    training_data: Path 
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/VaheC/CNN_Classifier.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [5]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [6]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config=val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 509s 1s/step - loss: 1.0119 - accuracy: 0.9008


In [7]:
os.getcwd()

'c:\\Users\\vchar\\OneDrive\\Desktop\\CNN_Classifier'